In [2]:
import pandas as pd
import json
from datetime import datetime

pd.options.display.max_rows = None
pd.options.display.max_columns = None

with open('data.json') as f:
    data = json.load(f)
dt = tournaments = pd.json_normalize(data['tournaments'])
dp = players = pd.json_normalize(data['players'])
dm = matches = pd.json_normalize(data['matches'])
dr = results = pd.json_normalize(data['results'])

In [3]:
dp = dp.set_index('name')
dt = dt.set_index('name')
dm = dm.set_index(['tournament','round'])
dt['date'] = dt.apply(lambda row: datetime.strptime(row['date'], '%d.%m.%Y').date(), axis=1)

In [4]:
def count(player, result):
    return _count(player, 'white', result) + _count(player, 'black', 1.0 - result)
def _count(player, side, result):
    return len(dm[(dm[side] == player) & (dm['result'] == result)])

dp['W'] = dp.apply(lambda row: count(row.name, 1.0), axis=1)
dp['D'] = dp.apply(lambda row: count(row.name, 0.5), axis=1)
dp['L'] = dp.apply(lambda row: count(row.name, 0.0), axis=1)
dp['M'] = dp['W'] + dp['D'] + dp['L']
dp['score'] = round( (dp['W'] +  dp['D']/2) * 7 / dp['M'], 2 )

In [5]:
def get_score(player):
    return 0.0 if player == 'No Opponent' else dp.loc[player,'score']

pairs = {}
for idx, m in dm.iterrows():
    sw = get_score(m['white'])
    sb = get_score(m['black'])
    key = (m['white'], m['black'])
    a,b,sb = pairs.get(key, (0,0,sb))
    pairs[key] = a + m['result'], b + 1,sb
    key = (m['black'], m['white'])
    a,b,sw = pairs.get(key, (0,0,sw))
    pairs[key] = a + 1.0-m['result'], b + 1,sw

d = [{'player':k[0],'opponent':k[1],'R':v[0],'M':v[1],'score':v[2]} for k,v in pairs.items()]
dd = duels = pd.DataFrame(d)
dd['P'] = round(dd['R'] / dd['M'] * 100,1)
dd = dd.set_index('player')

In [6]:
from glicko2 import Glicko2, WIN, DRAW, LOSS # WIN=1.0 DRAW=0.5 LOSS=0.0

def create_rating(label, initial_rating):
    env = Glicko2(tau=0.5)
    rank = {}
    for _,p in dp.iterrows():
        rtg = initial_rating(p.name)
        rd = 300 if rtg in [1000,1400,1600,1200] else 150
        rank[p.name] = env.create_rating(rtg, rd)

    for _,t in dt.sort_values('date').iterrows():
        for _,m in dm.loc[t.name].iterrows():
            if m['black'] == 'No Opponent':
                continue
            white = rank[m['white']]
            black = rank[m['black']]
            rank[m['white']] = env.rate(white, [(m['result'], black)])
            rank[m['black']] = env.rate(black, [(1.0 - m['result'], white)])
            if 'Lewandowski Mateusz' in [m['white'],m['black']]:
                r = rank['Lewandowski Mateusz']
                print(round(r.mu), round(r.phi), m['white'], m['black'], m['result'])

    dp[label] = dp.apply(lambda row: round(rank[row.name].mu), axis=1)

create_rating('ra', lambda name: 1400)
create_rating('rb', lambda name: dp.loc[name]['rating'])

1628 238 Pilak Wiktor Lewandowski Mateusz 0.0
1554 203 Lewandowski Mateusz Gabryszewski Hubert 0.0
1500 181 Huchwajda Katarzyna Lewandowski Mateusz 1.0
1526 170 Lewandowski Mateusz Kubik Michał 1.0
1565 156 Bielawny Maciej Lewandowski Mateusz 0.0
1610 144 Lewandowski Mateusz Domański Marcin 1.0
1664 134 Mikołajczyk Marcin Lewandowski Mateusz 0.0
1699 126 Borowiak Mateusz Lewandowski Mateusz 0.0
1657 119 Lewandowski Mateusz Truchel Tomasz 0.0
1681 114 Lewandowski Mateusz Kamiński Patryk 1.0
1735 111 Pawłowski Rafał Lewandowski Mateusz 0.0
1772 106 Lewandowski Mateusz Gabryszewski Hubert 1.0
1766 105 Mazurkiewicz Michał Lewandowski Mateusz 1.0
1770 101 Lewandowski Mateusz Grześkowiak Antoni 0.5
1751 100 Lewandowski Mateusz Waszkowska Sandra 0.5
1756 99 Ługowski Dominik Lewandowski Mateusz 0.0
1787 96 Lewandowski Mateusz Grześkowiak Antoni 1.0
1810 93 Gabryszewski Hubert Lewandowski Mateusz 0.0
1795 91 Lewandowski Mateusz Styliński Piotr 0.0
1814 89 Truchel Tomasz Lewandowski Mateusz 0.0


In [7]:
dp[dp['M'] > 21].sort_values('rb', ascending=False)[['title','rating','ra','rb','score']]
# dp.plot.scatter(x='rating',y='rb')
dp.sort_values('rb',ascending=False)

,title,club,birthdate,rating,W,D,L,M,score,ra,rb
name,,,,,,,,,,,
Licznerski Łukasz,IM,"LKS ""Parnas"" Stara Wieś",1995,2387,6,0,1,7,6.00,2113,2392
Sieciechowicz Marcin,IM,SzUKS Gostmat 83 Gostynin,1992,2280,67,5,5,77,6.32,2196,2302
Lubczyński Rafał,IM,"LKS ""Chrobry"" Gniezno",1988,2314,29,3,3,35,6.10,2120,2268
Mazurkiewicz Michał,CM,WKS Kopernik,1989,2063,183,9,18,210,6.25,2174,2165
Kulon Klaudia,IM,"LKS ""Wrzos"" Miedzyborów",1992,2281,4,0,3,7,4.00,1772,2099
Tofil Borys,K,,1991,2177,5,1,1,7,5.50,1826,2066
Małaczek Mateusz,II+,,1995,1880,66,11,7,84,5.96,2060,2041
Ulasevich Sergei,,,1991,2151,22,4,9,35,4.80,1919,2021
Paczka Patryk,I+,"ASSz ""Lipno"" Stęszew",1990,1894,24,2,9,35,5.00,1938,1981
